### Netbouncer implementation based on the [NSDI paper](https://www.usenix.org/system/files/nsdi19spring_tan_prepub.pdf)
The algorithm for Netbouncer is shown below and taken from the paper.



<img src="resources/alg.png" width="400px" />

### We are going to define a small system to verify our implementation and share it with the authors to validate it.

<img src="resources/test-system.jpg" width="500px" />

# Task 1 : Import libraries and enable/disable debug info

In [1]:
import numpy as np
DEBUG = False
def print_debug(line):
    if DEBUG:
        print(line)

# Task 2 : Define all functions

* initialization function to init components health using failure observation on paths (init_prob)
* cost function to check the fitting of X's health  (cost_func)
* coordinate descent algo as described in the paper (fit)

In [2]:
# def get_links(path_id):
#     return paths[path_id]

class NetBouncer:
    def __init__(self, paths, Lambda, numMaxIterations, eps):
        self.paths = paths
        self.num_paths = len(paths)
        self.numIterations = numMaxIterations
        self.eps = eps
        self.Lambda = Lambda
        self.YObs = None
        self.X = None
        self.components = None
        self.num_components = None
        self.set_components_from_paths()
    
    def set_components_from_paths(self,):
        components = [c for j in paths for c in paths[j] ]
        self.components =  np.unique(components)
        self.num_components = len(self.components)
    
    def init_prob(self,):
        X = np.zeros(self.num_components)
        for i in range(self.num_components):
            X[i] = np.mean([self.YObs[j] for j in range(YObs.shape[0]) if i in self.paths[j]])
        return X

    
    def cost_func(self):
        L = np.sum([(self.YObs[j] - np.prod([self.X[i] for i in self.paths[j]]))**2 for j in range(self.YObs.shape[0])]) + \
            self.Lambda*(np.sum((self.X[i]*(1-self.X[i])) for i in range(self.num_components)))
        return L

    def fit(self, YObs):
        self.YObs = YObs
        self.X =  self.init_prob() 
        print_debug("initialized X to {}".format(self.X))
        R = np.zeros(self.num_components)
        S = np.zeros(self.num_components)
        T = np.zeros(self.num_components)
        L_prev = self.cost_func()
        L = 0
        for k in range(self.numIterations):
            print_debug("iteration {}".format(k))
            for i in range(self.num_components):
                R[i] = 2*np.sum([np.prod([self.X[l] for l in self.paths[j] if l != i])**2 for j in range(self.YObs.shape[0])]) - \
                        2* self.Lambda
                S[i] = 2*np.sum([np.prod([self.X[l] for l in self.paths[j] if l != i])*self.YObs[j] for j in range(self.YObs.shape[0])]) - \
                        self.Lambda
                T = S//R
                print_debug("R is {}".format(R))
                print_debug("S is {}".format(S))
                if R[i] == 0:
                    if S[i] > 0:
                        self.X[i] = 1
                    else:
                        self.X[i] = 0
                elif R[i] > 0:
                    if T[i] > 1:
                        self.X[i] = 1
                    elif T[i] < 0:
                        self.X[i] = 0
                    else:
                        self.X[i] = T[i]
                else:
                    if T[i] > 1/2:
                        self.X[i] = 0
                    else:
                        self.X[i] = 1
            print_debug(self.X)
            L = self.cost_func()
            print_debug("L_prev: {}, L: {}".format(L_prev, L))
            if L_prev - L < self.eps: #eps:
                break
            L_prev = L
        return self.X

# Task 3: Specify system topology and paths

In [3]:
paths = {}
paths[0] = [0, 1, 2, 3]
paths[1] = [0, 1, 2, 4]
paths[2] = [5, 6, 2, 3]
paths[3] = [5, 6, 2, 4]

In [4]:
driver = NetBouncer(
    paths, 
    1.0, # lambda
    100, # numMaxIterations
    1e-3 # convergence threshold
    )

# Task 4: Evaluate different failures and failure modes

### specify YObs obtained on each path by calculating: ratio of successful requests/total requests on all paths

## Task 4.1 : Failstop component 3

In [5]:
YObs = np.array([0.0, 1.0, 0.0, 1.0 ]) 
driver.fit(YObs)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in floor_divide


array([1., 1., 1., 0., 1., 1., 1.])

## Task 4.2 : Paritally fail component 3 and 6 (both dropping 75% of the requests

In [6]:
YObs = np.array([0.25, 1.0, 0.0625, 0.25 ])
driver.fit(YObs)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in floor_divide


array([1., 1., 1., 0., 1., 1., 0.])

# Task 4.3: Partially fail component 3 and 6 at different thresholds (10 - 100%)

In [7]:
for c3 in range (10, 100, 10):
    for c6 in range(10, 100, 10):
        pc3 = c3/100.0
        pc6 = c6/100.0
        YObs =  np.array([1-pc3, 1.0, (1-pc3)*(1-pc6), (1-pc6)])
        componentHealth = driver.fit(YObs)
        if componentHealth[3] != 0 or componentHealth[6] != 0 :
            print("Health info pc3: {}, pc6:{} is {}".format(pc3, pc6, componentHealth))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in floor_divide


Health info pc3: 0.1, pc6:0.1 is [1. 1. 1. 1. 1. 1. 1.]
Health info pc3: 0.1, pc6:0.2 is [1. 1. 1. 1. 1. 1. 0.]
Health info pc3: 0.1, pc6:0.3 is [1. 1. 1. 1. 1. 1. 0.]
Health info pc3: 0.1, pc6:0.4 is [1. 1. 1. 1. 1. 1. 0.]
Health info pc3: 0.1, pc6:0.5 is [1. 1. 1. 1. 1. 1. 0.]
Health info pc3: 0.1, pc6:0.6 is [1. 1. 1. 1. 1. 1. 0.]
Health info pc3: 0.1, pc6:0.7 is [1. 1. 1. 1. 1. 1. 0.]
Health info pc3: 0.1, pc6:0.8 is [1. 1. 1. 1. 1. 1. 0.]
Health info pc3: 0.1, pc6:0.9 is [1. 1. 1. 1. 1. 1. 0.]
Health info pc3: 0.2, pc6:0.1 is [1. 1. 1. 1. 1. 1. 0.]
Health info pc3: 0.2, pc6:0.2 is [1. 1. 1. 1. 1. 1. 0.]
Health info pc3: 0.2, pc6:0.3 is [1. 1. 1. 1. 1. 1. 0.]
Health info pc3: 0.2, pc6:0.4 is [1. 1. 1. 1. 1. 1. 0.]
Health info pc3: 0.2, pc6:0.5 is [1. 1. 1. 1. 1. 1. 0.]
Health info pc3: 0.2, pc6:0.6 is [1. 1. 1. 1. 1. 1. 0.]
Health info pc3: 0.2, pc6:0.7 is [1. 1. 1. 1. 1. 1. 0.]
Health info pc3: 0.2, pc6:0.8 is [1. 1. 1. 1. 1. 1. 0.]
Health info pc3: 0.2, pc6:0.9 is [1. 1. 1. 1. 1.